In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from io import BytesIO
from PIL import Image
import os

In [2]:
product_pages = {'ps4_controller':'https://www.shutterstock.com/search/ps4+controller?mreleased=false&image_type=photo',
                 'iphone12_pro_max':'https://www.shutterstock.com/search/iphone+12+pro+max?mreleased=false&image_type=photo',
                 'jbl_charge3':'https://www.shutterstock.com/search/jbl+charge+3?mreleased=false&image_type=photo',
                 'yeezy_boost_350':'https://www.shutterstock.com/search/yeezy+boost+350?mreleased=false&image_type=photo',
                 'nintendo_switch':'https://www.shutterstock.com/search/nintendo+switch?mreleased=false&image_type=photo'}
products_folder = 'products_images'

In [3]:
def get_imgs_links(page_source):
    links_to_imgs = []
    
#    headers = {'user-agent':'Mozilla/5.0'}
#    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(page_source, 'html.parser')
    
    for i in range(100):
        try:
            img_div = soup.find('div', attrs={'data-automation':f'mosaic-grid-cell-{i}'})
            item_img = img_div.a.img
            img_link = item_img.get('src')
            links_to_imgs.append(img_link)
        except:
            continue
    return links_to_imgs

In [4]:
for product, url in product_pages.items():
    
    #initializing a selenium session
    browser = webdriver.Firefox()
    browser.implicitly_wait(5)
    #opening our target url with our selenium session
    browser.get(url)
    time.sleep(10)
    html = browser.find_element_by_tag_name('html')
    for i in range(5):
        html.send_keys(Keys.PAGE_DOWN)
        time.sleep(5)
    html.send_keys(Keys.END)
    time.sleep(5)
    
    img_links = get_imgs_links(browser.page_source)
    
    browser.close()
    
    if not os.path.exists(products_folder+'/'+product):
        os.makedirs(products_folder+'/'+product)
    
    index = 0
    for img_link in img_links:
        img_response = requests.get(img_link)
        img = Image.open(BytesIO(img_response.content))
        img.save(products_folder + '/' + product +'/' + product + '_' + str(index) + '.jpg')
        index += 1